#Project_Draft_K_Test_M1_Unseen 

This notebook trains the classification model on all of the training data, and then tests it on data that has never been seen before.


##Set up Drive

In [1]:
#authorize Colab to access Drive
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


##Imports and Constants

In [13]:
import gensim
import csv
import os
import pickle
import tabulate
import numpy as np
import re
import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize
from collections import defaultdict
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.matutils import sparse2full


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##Set up spaCy

code adapted from https://heartbeat.fritz.ai/nlp-chronicles-intro-to-spacy-34949f1bc118#08c8

spaCy documentation  https://spacy.io/usage

spaCy POS tags
https://spacy.io/api/annotation#pos-tagging

In [3]:
pip install -U spaCy

     |████████████████████████████████| 10.4MB 4.8MB/s 
     |████████████████████████████████| 2.2MB 37.5MB/s 
     |████████████████████████████████| 122kB 48.8MB/s 
     |████████████████████████████████| 3.7MB 36.8MB/s 
  Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninstalled blis-0.2.4
  Found existing installation: thinc 7.0.8
    Uninstalling thinc-7.0.8:
      Successfully uninstalled thinc-7.0.8
  Found existing installation: spacy 2.1.9
    Uninstalling spacy-2.1.9:
      Successfully uninstalled spacy-2.1.9


In [4]:
!python -m spacy download en

     |████████████████████████████████| 12.0MB 817kB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-cp36-none-any.whl size=12011741 sha256=c0f30adf317a25d156723e154920d7cf560d2aac9a0576e0c9179f1867e415af
  Stored in directory: /tmp/pip-ephem-wheel-cache-gn0jzn1l/wheels/6a/47/fb/6b5a0b8906d8e8779246c67d4658fd8a544d4a03a75520197a
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.1.0
    Uninstalling en-core-web-sm-2.1.0:
      Successfully uninstalled en-core-web-sm-2.1.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
import spacy
from spacy import displacy
nlp=spacy.load('en')

## Define a function to represent the high-level syntax of a sentence

It should extract the top syntactic levels of the parse tree, excluding the leaves (tokens). 

The syntactic abstraction of the sentence is what will be used for classification.

In [0]:
def parse(text):
# create a spaCy document instance from the text 
    doc = nlp(text)
# find the root of the parse tree on which all other tokens depend
    root = [token for token in doc if token.head == token][0]

# starting from the root, find the syntactic tags of the first two levels of the parse tree.
# using tags instead of pos for greater differentiation of verb types.
    parsed = ['ROOT_self_' + root.tag_]
    for s in root.lefts:
        parsed.extend(['LEFT_' + s.dep_ +'_' + s.tag_])

    for s in root.rights:
        parsed.extend(['RIGHT_' + s.dep_ + '_' + s.tag_])
        
    return parsed

In [0]:
# demonstrate the function's output
parse('I want to be a clone.')

['ROOT_self_VBP', 'LEFT_nsubj_PRP', 'RIGHT_xcomp_VB', 'RIGHT_punct_.']

## Define Vectorizer Class

based on example distributed in course

In [0]:
class GensimTfidfVectorizer(BaseEstimator, TransformerMixin):

    def __init__(self, dirpath=".", tofull=False):
        """
        Pass in a directory that holds the lexicon in corpus.dict and the
        TFIDF model in tfidf.model (for now).

        Set tofull = True if the next thing is a Scikit-Learn estimator
        otherwise keep False if the next thing is a Gensim model.
        """
        self._lexicon_path = os.path.join(dirpath, "corpus.dict")
        self._tfidf_path = os.path.join(dirpath, "tfidf.model")

        self.lexicon = None
        self.tfidf = None
        self.tofull = tofull

        self.load()

    def load(self):

        if os.path.exists(self._lexicon_path):
            self.lexicon = Dictionary.load(self._lexicon_path)

        if os.path.exists(self._tfidf_path):
            self.tfidf = TfidfModel().load(self._tfidf_path)

    def save(self):
        self.lexicon.save(self._lexicon_path)
        self.tfidf.save(self._tfidf_path)

    def fit(self, documents, labels=None):
        self.lexicon = Dictionary(documents)
        self.tfidf = TfidfModel([self.lexicon.doc2bow(doc) for doc in documents], id2word=self.lexicon)
        self.save()
        return self

    def transform(self, documents):
        def generator():
            for document in documents:
                vec = self.tfidf[self.lexicon.doc2bow(document)]
                if self.tofull:
                    yield sparse2full(vec, len(self.lexicon))
                else:
                    yield vec
        return list(generator())


#Main section

## Read in the Pickled Training Data

Non-actions are duplicated to approximate the same number as the Actions, so the training data is balanced.

In [0]:
# location of pickle files
actions_file = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/Climate Change Docs - Actions.pkl'
non_actions_file = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/Climate Change Docs - Non-Actions.pkl'

#initialize accumulators
corpus = []
labels = []

with open(actions_file, 'r', encoding="utf8", errors='ignore') as f:
    reader = csv.reader(f, delimiter=' ')
    for row in reader:
        corpus.append(row)
        labels.append('action')

with open(non_actions_file, 'r', encoding="utf8", errors='ignore') as f:
    reader = csv.reader(f, delimiter=' ')
    for row in reader:
        corpus.append(row)
        corpus.append(row)
        labels.append('non_action')
        labels.append('non_action')


In [9]:
# show the training data
for i in range(3):
    print(labels[i], corpus[i])

print("...")    

for i in range(-3, 0, 1): 
    print(labels[i], corpus[i])   
    

action ['ROOT_self_VB', 'LEFT_aux_VB', 'LEFT_nsubj_NN', 'RIGHT_dobj_NN', 'RIGHT_advcl_VB', 'RIGHT_punct_.']
action ['ROOT_self_VB', 'LEFT_neg_RB', 'RIGHT_dobj_NN', 'RIGHT_prep_IN', 'RIGHT_advcl_VB', 'RIGHT_punct_.']
action ['ROOT_self_VB', 'RIGHT_dobj_NN', 'RIGHT_advcl_VB', 'RIGHT_punct_.']
...
non_action ['ROOT_self_VB', 'RIGHT_ccomp_VBZ', 'RIGHT_punct_.']
non_action ['ROOT_self_VBN', 'LEFT_nsubjpass_NNS', 'LEFT_auxpass_VBD', 'LEFT_neg_RB', 'RIGHT_xcomp_VB', 'RIGHT_punct_,', 'RIGHT_cc_CC', 'RIGHT_conj_VBD']
non_action ['ROOT_self_VBN', 'LEFT_nsubjpass_NNS', 'LEFT_auxpass_VBD', 'LEFT_neg_RB', 'RIGHT_xcomp_VB', 'RIGHT_punct_,', 'RIGHT_cc_CC', 'RIGHT_conj_VBD']


##Train a Logistic Regression model on the whole training corpus

In [10]:
# where to save the models
%cd '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/FinalModel'

# use the whole training corpus
docs=corpus
labs=labels

# fit the Vectorizer to the training data and save it
v=GensimTfidfVectorizer('.', True) 
v.fit(docs)

# use the Vectorizer to transform the training data
vecs=v.transform(docs)

# fit the Classifier to the vectorized training data and save it
m=LogisticRegression()
m.fit(vecs, labels)
pickle.dump(m, open('LRClassifier.model', 'wb'))


/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/FinalModel


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [11]:
# show the vectors
print(vecs[0])
print("...")    
print(vecs[-1])

[0.7705148  0.2970811  0.4859179  0.25309247 0.02639841 0.13107584
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0. 

## Parse Reserved Test Data 

Using the text extracted by pdfminer

In [0]:
# manually construct a list of eligible files that have not been examined yet, 
# and may contain both actions and non-actions that have never been seen before

test_files = [
  'coastal_flooded_land_guidelines.txt',
  'En56-226-2008-eng.txt',
  'env-yukon-state-play-analysis-climate-change-impacts-adaptation.txt',
  'FBC_WaterGuide_FINAL.txt',
  'final_climate_change_and_health_backgrounder_overview.txt',
  'Guidebook-2016.txt',
  'HP5-122-2017-eng.txt',
  'landuse-e.txt',
  'preparedbc_flood_information_for_homeowners_and_home_buyers_2018.txt',
  'protect-your-home-from-basement-flooding.txt',
  'Protect_Your_Home_From_Flooding_Brochure.txt',
  'sea_dike_guidelines.txt',
  'slr-primer.txt',
  'Synthesis_Eng.txt',
  'Urban_Forests_Guide.txt',
  'Vancouver-Climate-Change-Adaptation-Strategy-2012-11-07.txt',
  'WCEL_climate_change_FINAL.txt'
  ]

test_path = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/ClimateChangeDocs_pdfminer'

# initialize variables
candidates = [] 
clean_sents = [] 
sent_files = []  
counter = 0

# read each file in the list and clean up the raw text
for file in test_files:
    path_to_txt = os.path.join(test_path, file)
    with open(path_to_txt, encoding='utf8') as f:
        raw_text=f.read()
        
        for sent in sent_tokenize(raw_text): # nltk.sent_tokenize is better at recognizing sentences across multiple lines than spaCy is
            #print("Raw: ", sent)
            sent = re.sub(r'\s+', ' ', sent) # replace multiple whitespace characters with a single space
            sent = re.sub(r'[^\x00-\x7F]', '_', sent) # replace non-ascii characters with underscore
            #print("Clean: ", sent)

#           apply spaCy nlp processing to cleaned-up sentence
            sent = nlp(sent)
            #print("spaCy: ", sent.text)

#           skip sentences with 3 or fewer words, as these are not likely to be 'actions a citizen can do'
            if len(word_tokenize(sent.text)) > 3: 
                clean_sents.append(sent.text)
                sent_files.append(file)
                candidates.append(parse(sent.text))
                counter += 1

#stop when 50,000 sentences found, because that is enough for a human to look at
    if counter > 50000:
        break

In [0]:
# show the parsed candidates
print("Found {} candidate sentences   \n".format(counter))
for i in range(0, 3):
    print("Clean: ", clean_sents[i])
    print("Parsed: ", candidates[i])
    print("File: ", sent_files[i], '\n')

print("...")    

for i in range(-3, 0):
    print("Clean: ", clean_sents[i])
    print("Parsed: ", candidates[i])
    print("File: ", sent_files[i], '\n')


Found 14278 candidate sentences   

Clean:   Process Infrastructure Ports, Marine & Offshore Project No.
Parsed:  ['ROOT_self_NNPS', 'LEFT_compound_NN', 'LEFT_compound_NNP', 'RIGHT_punct_,', 'RIGHT_conj_NNP']
File:  coastal_flooded_land_guidelines.txt 

Clean:  143111 Revision Number 0 BC Ministry of Environment Climate Change Adaption Guidelines for Sea Dikes and Coastal Flood Hazard Land Use Guidelines for Management of Coastal Flood Hazard Land Use 27 January 2011 DISCLAIMER: This document is for the private information and benefit only of the client for whom it was prepared and for the particular purpose previously advised to Ausenco Sandwell.
Parsed:  ['ROOT_self_NNP', 'LEFT_nsubj_NNPS', 'RIGHT_dobj_NNPS']
File:  coastal_flooded_land_guidelines.txt 

Clean:  The contents of this document are not to be relied upon or used, in whole or in part, by or for the benefit of others without prior adaptation and specific written verification by Ausenco Sandwell.
Parsed:  ['ROOT_self_VBP', '

In [0]:
# save the parsed candidates
out_path = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/candidates_pdfminer.pkl'
with open(out_path, 'w') as outfile:
     for candidate in candidates:
         outfile.writelines(' '.join(candidate))
         outfile.write('\n')

out_path = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/candidates_text_pdfminer.pkl'
with open(out_path, 'w') as outfile:
     for sent in clean_sents:
         outfile.writelines(sent)
         outfile.write('\n')

out_path = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/candidate_files_pdfminer.pkl'
with open(out_path, 'w') as outfile:
     for filename in sent_files:
         outfile.writelines(filename)
         outfile.write('\n')

## Find actions using a hard-coded rule on the parses

In [0]:
# Look for sentences that start with 'ROOT_self_VB RIGHT_dobj_NN*'
for i in range(len(candidates)):
  if candidates[i][0] == 'ROOT_self_VB' and candidates[i][1].startswith('RIGHT_dobj_NN'):
    print(i, clean_sents[i], candidates[i], sent_files[i])


197 Use water-resistant materials and construction as appropriate. ['ROOT_self_VB', 'RIGHT_dobj_NNS', 'RIGHT_prep_IN', 'RIGHT_punct_.'] coastal_flooded_land_guidelines.txt
407 Identify the focus and objectives of a SAM initiative Step 2. ['ROOT_self_VB', 'RIGHT_dobj_NN', 'RIGHT_punct_.'] En56-226-2008-eng.txt
408 Assess present status and trends. ['ROOT_self_VB', 'RIGHT_dobj_NN', 'RIGHT_punct_.'] En56-226-2008-eng.txt
410 Develop a vision of the future. ['ROOT_self_VB', 'RIGHT_dobj_NN', 'RIGHT_punct_.'] En56-226-2008-eng.txt
456 Examine current development challenges, planning principles and capacities b. ['ROOT_self_VB', 'RIGHT_dobj_NNS', 'RIGHT_punct_,', 'RIGHT_conj_VBG'] En56-226-2008-eng.txt
459 Identify future development priorities based on the principles of local sustainability and community planning b. Assess impacts of climate change and the potential for adaptation and mitigation within community goals STEP 4 Set trajectories to meet priorities. ['ROOT_self_VB', 'RIGHT_dobj_N

In [0]:
# Look at a sample of sentences that do NOT start with 'ROOT_self_VB RIGHT_dobj_NN*'
for i in range(0, len(candidates), 200):
  if candidates[i][0] == 'ROOT_self_VB' and candidates[i][1].startswith('RIGHT_dobj_NN'):
      continue

  else:    
    print(i, clean_sents[i], candidates[i], sent_files[i])


0  Process Infrastructure Ports, Marine & Offshore Project No. ['ROOT_self_NNPS', 'LEFT_compound_NN', 'LEFT_compound_NNP', 'RIGHT_punct_,', 'RIGHT_conj_NNP'] coastal_flooded_land_guidelines.txt
200 Such infrastructure should be designed and constructed to remain operational during floods. ['ROOT_self_VBN', 'LEFT_nsubjpass_NN', 'LEFT_aux_MD', 'LEFT_auxpass_VB', 'RIGHT_cc_CC', 'RIGHT_conj_VBN', 'RIGHT_punct_.'] coastal_flooded_land_guidelines.txt
400 We are especially grateful to the Government of Canada_s Climate Change Impacts and Adaptation Program (CCIAP) (NRCan) for financial support for this Guidebook and the preparatory workshop for this publication (CCIAP project A-1439). ['ROOT_self_VBP', 'LEFT_nsubj_PRP', 'RIGHT_acomp_JJ'] En56-226-2008-eng.txt
600 The voices in my head that don_t want to be seen to always be a bother, that want to be liked, that are also tired and panicked at the enormity of the global warming issue, convinced me, against my true better judgment, to be silent.

###Discussion:

Almost all the sentences found by the first cell can be considered actions of some kind, although some are actions that only trusted authorities could do.

The sentences found by the second cell do contain some actions (for example #1000), but they seem to be only a small fraction of the sample.

Therefore it seems like the hard-coded rule is a viable way of screening for actions in text documents.





##Predict whether the sentences are actions using an ML model

Training data consists of about 1600 sentences that have been manually extracted from the pdf corpus and labelled as actions or non-actions. 

Each sentence is represented by its parse tokens, which are treated as words.

Parses are vectorized using TF-IDF, and then a Logistic Regression model is trained on them.


In [0]:
# where to get the models from if they were not already in memory
#%cd '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/FinalModel'
#v = GensimTfidfVectorizer('.', True)
#m = pickle.load(open('LRClassifier.model', 'rb'))

# test data has been previously read in and parsed
docs=candidates

# Vectorizer
v=GensimTfidfVectorizer('.', True) 
vecs=v.transform(docs)

# Predict
result = m.predict(vecs)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# Print only the sentences classified as actions
if 'action' in result:
    print("Found actions ...")

    for i in range(len(result)):
        if result[i] == 'action':
            print("{}:  {}".format(i, clean_sents[i]))

else:            
    print("No actions found.")

Found actions ...
0:   Process Infrastructure Ports, Marine & Offshore Project No.
7:  Revision Status Revision Date Description Contributors Reviewer FirstName LastName Position Title FirstName LastName Position Title Approver FirstName LastName For Internal Information/Discussion For Internal Information/Discussion For Stakeholder Meeting HR/RA/JSR DR/HR/JSR DR/HR/JSR For Client Use DR/JSR Final Issue DR/JSR/Client Client internal Client JM JM A 31 March 2010 A2-A7 various 20 June 2020 31 October 2010 27 January 2011 A8 B 0 Signature Position Title Rev: 0 Date: 27 January 2011 Project No: 143111: BC Ministry of Environment/ Climate Change Adaption Guidelines for Sea Dikes and Coastal Flood Hazard Land Use 1 1 1 2 2 2 2 2 5 5 6 7 9 11 11 13 14 16 18 20 20 20 21 21 21 21 21 22 22 Contents Introduction and Application of This Document General Acknowledgment Background Scope Reference Documents Definitions Updated Definitions Climate Change Impacts on Coastal Land Use Management Incremen

In [0]:
# Print a sample of sentences with their labels
for i in range(0, 5000, 100):
    print("{}:  {}".format(result[i], clean_sents[i]))


action:   Process Infrastructure Ports, Marine & Offshore Project No.
action:  This approach will minimize the initial costs of considering SLR, and the future costs of adaptation.
action:  Such infrastructure should be designed and constructed to remain operational during floods.
non_action:  1.16 Sea Dike System A system of: dikes, dunes, berms or natural shorelines that provide a similar function; and associated engineering works (e.g., tidal gates, outfalls, outlet structures, seawalls, quay walls, ramps, adjacent building features, etc.)
non_action:  We are especially grateful to the Government of Canada_s Climate Change Impacts and Adaptation Program (CCIAP) (NRCan) for financial support for this Guidebook and the preparatory workshop for this publication (CCIAP project A-1439).
non_action:  Over the years since the United Nations Framework Convention on Climate Change was first signed in Rio de Janeiro in 1992, there have been a number of efforts led by national governments atte

###Discussion

The ML model's assignment of action and non_action labels to unseen test data seems little better than random. 

This is disappointing because initial scores were in the 80% range when cross-validating the model on the training data.

It may be that the model is overfitted to the small training set, since there are 225 features and only 1208 unique training examples. 

Also, since the training data were hand-picked to represent examples of actions and non_actions, it may be that they represent unusually extreme examples of actions and non-actions. When trained on extreme ends of a spectrum, the classifier might have more difficulty distinguishing data that falls closer to the middle of the spectrum.

It might be worthwhile to continue experiments with an ML approach. 
In that case, it would be useful to
*   using more training data
*   extract the training data with a random picker, and have the human only label it
*   try different classifier models


##Logistic Regression with Truncated SVD

## Parse Reserved Test Data 

Using the text extracted by pdfminer

In [0]:
# manually construct a list of eligible files that have not been examined yet, 
# and may contain both actions and non-actions that have never been seen before

test_files = [
  'coastal_flooded_land_guidelines.txt',
  'En56-226-2008-eng.txt',
  'env-yukon-state-play-analysis-climate-change-impacts-adaptation.txt',
  'FBC_WaterGuide_FINAL.txt',
  'final_climate_change_and_health_backgrounder_overview.txt',
  'Guidebook-2016.txt',
  'HP5-122-2017-eng.txt',
  'landuse-e.txt',
  'preparedbc_flood_information_for_homeowners_and_home_buyers_2018.txt',
  'protect-your-home-from-basement-flooding.txt',
  'Protect_Your_Home_From_Flooding_Brochure.txt',
  'sea_dike_guidelines.txt',
  'slr-primer.txt',
  'Synthesis_Eng.txt',
  'Urban_Forests_Guide.txt',
  'Vancouver-Climate-Change-Adaptation-Strategy-2012-11-07.txt',
  'WCEL_climate_change_FINAL.txt'
  ]

test_path = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/ClimateChangeDocs_pdfminer'

# initialize variables
candidates = [] 
clean_sents = [] 
sent_files = []  
counter = 0

# read each file in the list and clean up the raw text
for file in test_files:
    path_to_txt = os.path.join(test_path, file)
    with open(path_to_txt, encoding='utf8') as f:
        raw_text=f.read()
        
        for sent in sent_tokenize(raw_text): # nltk.sent_tokenize is better at recognizing sentences across multiple lines than spaCy is
            #print("Raw: ", sent)
            sent = re.sub(r'\s+', ' ', sent) # replace multiple whitespace characters with a single space
            sent = re.sub(r'[^\x00-\x7F]', '_', sent) # replace non-ascii characters with underscore
            #print("Clean: ", sent)

#           apply spaCy nlp processing to cleaned-up sentence
            sent = nlp(sent)
            #print("spaCy: ", sent.text)

#           skip sentences with 3 or fewer words, as these are not likely to be 'actions a citizen can do'
            if len(word_tokenize(sent.text)) > 3: 
                clean_sents.append(sent.text)
                sent_files.append(file)
                candidates.append(parse(sent.text))
                counter += 1

#stop when 50,000 sentences found, because that is enough for a human to look at
    if counter > 50000:
        break

In [17]:
# show the parsed candidates
print("Found {} candidate sentences   \n".format(counter))
for i in range(0, 3):
    print("Clean: ", clean_sents[i])
    print("Parsed: ", candidates[i])
    print("File: ", sent_files[i], '\n')

print("...")    

for i in range(-3, 0):
    print("Clean: ", clean_sents[i])
    print("Parsed: ", candidates[i])
    print("File: ", sent_files[i], '\n')


Found 14278 candidate sentences   

Clean:   Process Infrastructure Ports, Marine & Offshore Project No.
Parsed:  ['ROOT_self_NNPS', 'LEFT_compound_NN', 'LEFT_compound_NNP', 'RIGHT_punct_,', 'RIGHT_conj_NNP']
File:  coastal_flooded_land_guidelines.txt 

Clean:  143111 Revision Number 0 BC Ministry of Environment Climate Change Adaption Guidelines for Sea Dikes and Coastal Flood Hazard Land Use Guidelines for Management of Coastal Flood Hazard Land Use 27 January 2011 DISCLAIMER: This document is for the private information and benefit only of the client for whom it was prepared and for the particular purpose previously advised to Ausenco Sandwell.
Parsed:  ['ROOT_self_NNP', 'LEFT_nsubj_NNPS', 'RIGHT_dobj_NNPS']
File:  coastal_flooded_land_guidelines.txt 

Clean:  The contents of this document are not to be relied upon or used, in whole or in part, by or for the benefit of others without prior adaptation and specific written verification by Ausenco Sandwell.
Parsed:  ['ROOT_self_VBP', '

In [0]:
# save the parsed candidates
out_path = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/candidates_pdfminer.pkl'
with open(out_path, 'w') as outfile:
     for candidate in candidates:
         outfile.writelines(' '.join(candidate))
         outfile.write('\n')

out_path = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/candidates_text_pdfminer.pkl'
with open(out_path, 'w') as outfile:
     for sent in clean_sents:
         outfile.writelines(sent)
         outfile.write('\n')

out_path = '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/candidate_files_pdfminer.pkl'
with open(out_path, 'w') as outfile:
     for filename in sent_files:
         outfile.writelines(filename)
         outfile.write('\n')

##Train a Logistic Regression model on the whole training corpus after  Truncated SVD applied

In [24]:
# where to save the models
%cd '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/FinalModel'

# use the whole training corpus
docs=corpus
labs=labels

# fit the Vectorizer to the training data and save it
v=GensimTfidfVectorizer('.', True) 
v.fit(docs)

# use the Vectorizer to transform the training data
vecs=v.transform(docs)

# Truncated SVD
t=TruncatedSVD(n_components=5)
vecs_reduced=t.fit_transform(vecs)

# fit the Classifier to the vectorized training data and save it
m=LogisticRegression()
m.fit(vecs_reduced, labels)
pickle.dump(m, open('LRClassifier.model', 'wb'))


/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/FinalModel


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [25]:
# show the vectors
print(vecs_reduced[0])
print("...")    
print(vecs_reduced[-1])

[ 0.3161857   0.13685243 -0.03990754  0.14029232 -0.115687  ]
...
[ 0.1383982  -0.18315855 -0.13795115 -0.10512668  0.02034208]


##Predict whether the sentences are actions using an ML model

Training data consists of about 1600 sentences that have been manually extracted from the pdf corpus and labelled as actions or non-actions. 

Each sentence is represented by its parse tokens, which are treated as words.

Parses are vectorized using TF-IDF, and then a Logistic Regression model is trained on them.


In [26]:
# where to get the models from if they were not already in memory
#%cd '/gdrive/My Drive/Colab Notebooks/3666 ANLP/Project/FinalModel'
#v = GensimTfidfVectorizer('.', True)
#m = pickle.load(open('LRClassifier.model', 'rb'))

# test data has been previously read in and parsed
docs=candidates

# Vectorizer
v=GensimTfidfVectorizer('.', True) 
vecs=v.transform(docs)

# Truncated SVD
t=TruncatedSVD(n_components=5)
vecs_reduced=t.fit_transform(vecs)

# Predict
result = m.predict(vecs_reduced)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [27]:
# Print only the sentences classified as actions
if 'action' in result:
    print("Found actions ...")

    for i in range(len(result)):
        if result[i] == 'action':
            print("{}:  {}".format(i, clean_sents[i]))

else:            
    print("No actions found.")

Output hidden; open in https://colab.research.google.com to view.

In [28]:
# Print a sample of sentences with their labels
for i in range(0, 5000, 100):
    print("{}:  {}".format(result[i], clean_sents[i]))


action:   Process Infrastructure Ports, Marine & Offshore Project No.
action:  This approach will minimize the initial costs of considering SLR, and the future costs of adaptation.
action:  Such infrastructure should be designed and constructed to remain operational during floods.
action:  1.16 Sea Dike System A system of: dikes, dunes, berms or natural shorelines that provide a similar function; and associated engineering works (e.g., tidal gates, outfalls, outlet structures, seawalls, quay walls, ramps, adjacent building features, etc.)
non_action:  We are especially grateful to the Government of Canada_s Climate Change Impacts and Adaptation Program (CCIAP) (NRCan) for financial support for this Guidebook and the preparatory workshop for this publication (CCIAP project A-1439).
action:  Over the years since the United Nations Framework Convention on Climate Change was first signed in Rio de Janeiro in 1992, there have been a number of efforts led by national governments attempting t

###Discussion

The ML model's assignment of action and non_action labels to unseen test data seems little better than random. 

This is disappointing because initial scores were in the 80% range when cross-validating the model on the training data.

It may be that the model is overfitted to the small training set, since there are 225 features and only 1208 unique training examples. 

Also, since the training data were hand-picked to represent examples of actions and non_actions, it may be that they represent unusually extreme examples of actions and non-actions. When trained on extreme ends of a spectrum, the classifier might have more difficulty distinguishing data that falls closer to the middle of the spectrum.

It might be worthwhile to continue experiments with an ML approach. 
In that case, it would be useful to
*   using more training data
*   extract the training data with a random picker, and have the human only label it
*   try different classifier models
